In [7]:
import pandas as pd
import numpy as np
import pickle

import nltk

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
df = pd.read_pickle('childrens_book_df_with_sounds.pkl')

In [5]:
df.head()

,Title,Author,Chapter,NLP_Text,Readable_Text,Readable_Passage_Len,oo,ou,ow,oi,...,ce,ci,cy,ge,dge,gi,gy,kn,wr,gn
0,Prince Prigio,Andrew Lang,I,lcb chapter heading picture p1jpg rcb fairies ...,How the Fairies were not Invited to Court. Onc...,5500,0.002727,0.008182,0.001818,0.000364,...,0.003636,0.000000,0.000000,0.000364,0.000000,0.000545,0.000000,0.000545,0.000000,0.000545
1,Prince Prigio,Andrew Lang,II,lcb chapter heading picture p9jpg rcb prince p...,"Prince Prigio and his Family. Well, the little...",3210,0.004050,0.010280,0.004984,0.000000,...,0.005296,0.000623,0.000000,0.001558,0.000000,0.001869,0.000000,0.000623,0.000312,0.000312
2,Prince Prigio,Andrew Lang,III,firedrake people like prigio dear papa king gr...,About the Firedrake. Of all the people who did...,8008,0.001873,0.012488,0.002997,0.000874,...,0.003621,0.000125,0.000125,0.002498,0.000000,0.002872,0.000000,0.000500,0.000250,0.000250
3,Prince Prigio,Andrew Lang,IV,prince prigio deserted everybody meanwhile pri...,How Prince Prigio was Deserted by Everybody. M...,4607,0.001954,0.008465,0.004124,0.000434,...,0.006295,0.000434,0.000000,0.000868,0.000217,0.002388,0.000217,0.000868,0.000217,0.000217
4,Prince Prigio,Andrew Lang,V,prince prigio found garret prince walked room ...,What Prince Prigio found in the Garret. The pr...,1412,0.007790,0.005666,0.003541,0.000708,...,0.004249,0.000000,0.000708,0.000000,0.000000,0.000708,0.000000,0.000000,0.000708,0.000000


In [10]:
#clean further
df['NLP_Text'] = df['NLP_Text'].str.replace('\d+', '') #remove numbers


<ipython-input-10-8d54d74d6383>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['NLP_Text'] = df['NLP_Text'].str.replace('\d+', '') #remove numbers


In [17]:
df.iloc[1, 3]

'lcb chapter heading picture pjpg rcb prince prigio family well little prince grew think told prigio well think clever argued nurse soon could speak soon argued like washed soap got eyes however told pores skin could healthy washed ceased resist reasonable argued father see kings rich beggars poor king little greedy poached eggs plumcake afternoon tea many persons went without dinner king surprised hurt remarks boxed prince ears saying teach clever lad remembered awful curse oldest fairy sorry rudeness queen prince ears boxed said force argument king went away rage lcb prigio reading book pjpg rcb indeed tell prince hated would go kitchen show cook make soup would visit poor people cottage teach make beds make plumpudding turniptops venison cutlets rusty bacon showed fencingmaster fence professional cricketer bowl instructed ratcatcher breeding terriers set sums chancellor exchequer assured astronomer royal sun go round earth part believe young ladies court disliked dancing spite good 

In [18]:
#count vectorize
count_vectorizer = CountVectorizer(stop_words = 'english')
words_count_vectorize = count_vectorizer.fit_transform(df.NLP_Text)
words_count_vectorize

<866x49878 sparse matrix of type '<class 'numpy.int64'>'
	with 600910 stored elements in Compressed Sparse Row format>

In [19]:
#create new word matrix as df
features = count_vectorizer.get_feature_names()
df_words_count_vec = pd.DataFrame(words_count_vectorize.toarray(), columns=features)
df_words_count_vec

,aa,aaamen,aaron,ab,aback,abacus,abaft,abandon,abandoned,abandoning,...,zulamith,zulbazan,zylland,zytomar,ælueva,æneas,æsop,æsthetic,ætna,úprincess
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#also try tf-idf
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')
words_tfidf = tfidf_vectorizer.fit_transform(df.NLP_Text)
words_tfidf

#new new matrix as df
features = tfidf_vectorizer.get_feature_names()
df_words_tfidf = pd.DataFrame(words_tfidf.toarray(), columns=features)
df_words_tfidf

,aa,aaamen,aaron,ab,aback,abacus,abaft,abandon,abandoned,abandoning,...,zulamith,zulbazan,zylland,zytomar,ælueva,æneas,æsop,æsthetic,ætna,úprincess
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#nmf / count vec topics
nmf = NMF(n_components=10, random_state=42)

# fit the transfomed content with NMF
nmf.fit(words_count_vectorize)

# display the top 20 words for 10 topics
for index,topic in enumerate(nmf.components_):
    print(f"The top 20 words for topic # {index}")
    print([count_vectorizer.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print("\n")

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


The top 20 words for topic # 0
['palace', 'day', 'gold', 'fairy', 'long', 'good', 'shall', 'got', 'saw', 'time', 'away', 'great', 'prince', 'queen', 'went', 'came', 'princess', 'king', 'little', 'said']


The top 20 words for topic # 1
['going', 'home', 'looked', 'good', 'oh', 'time', 'think', 'eyes', 'came', 'thought', 'aunt', 'mrs', 'went', 'miss', 'know', 'like', 'little', 'old', 'said', 'nt']


The top 20 words for topic # 2
['thought', 'time', 'gilbert', 'good', 'oh', 'going', 'old', 'mr', 'little', 'miss', 'diana', 'know', 'think', 'davy', 'marilla', 'like', 'mrs', 'said', 'nt', 'anne']


The top 20 words for topic # 3
['know', 'great', 'jimmy', 'time', 'way', 'long', 'chuck', 'johnny', 'big', 'billy', 'reddy', 'frog', 'green', 'fox', 'grandfather', 'rabbit', 'nt', 'old', 'mr', 'little']


The top 20 words for topic # 4
['road', 'holy', 'boy', 'time', 'head', 'away', 'great', 'came', 'mowgli', 'sahib', 'good', 'like', 'know', 'mahbub', 'old', 'thy', 'thee', 'little', 'thou', 'sai

In [24]:
# nmf -> tf-idf=
nmf = NMF(n_components=10, random_state=42)

# fit the transfomed content with NMF
nmf.fit(words_tfidf)

# display the result
for index,topic in enumerate(nmf.components_):
    print(f"The top 20 words for topic # {index}")
    print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print("\n")

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


The top 20 words for topic # 0
['came', 'know', 'away', 'aunt', 'went', 'story', 'girl', 'felicity', 'good', 'mother', 'mr', 'mrs', 'beth', 'old', 'amy', 'laurie', 'like', 'little', 'said', 'nt']


The top 20 words for topic # 1
['big', 'gun', 'river', 'hunting', 'nt', 'terrible', 'knew', 'pond', 'hunters', 'hounds', 'antlers', 'jay', 'green', 'stranger', 'deer', 'forest', 'paddy', 'sammy', 'hunter', 'lightfoot']


The top 20 words for topic # 2
['henyard', 'dinner', 'henhouse', 'fat', 'little', 'mouse', 'hound', 'meadow', 'hens', 'rabbit', 'brown', 'nt', 'farmer', 'coyote', 'old', 'danny', 'bowser', 'fox', 'granny', 'reddy']


The top 20 words for topic # 3
['jane', 'little', 'know', 'barry', 'think', 'like', 'oh', 'lynde', 'captain', 'miss', 'cornelia', 'mrs', 'said', 'matthew', 'gilbert', 'leslie', 'diana', 'nt', 'marilla', 'anne']


The top 20 words for topic # 4
['like', 'joanna', 'shall', 'hand', 'shoreby', 'cried', 'good', 'returned', 'richard', 'shelton', 'lawless', 'nay', 'mat

In [25]:
#lda on count vec
LDA = LatentDirichletAllocation(n_components = 10, n_jobs = -2, random_state = 42)

# fit the transfomed content with LDA
LDA.fit(words_count_vectorize)

# display the result
for index,topic in enumerate(LDA.components_):
    print(f"The top 20 words for topic # {index}")
    print([count_vectorizer.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print("\n")

The top 20 words for topic # 0
['young', 'came', 'new', 'life', 'ye', 'shall', 'good', 'great', 'heart', 'time', 'hand', 'people', 'mowgli', 'mr', 'long', 'alan', 'like', 'little', 'said', 'old']


The top 20 words for topic # 1
['poor', 'shall', 'nt', 'way', 'silver', 'long', 'came', 'began', 'time', 'great', 'captain', 'little', 'cried', 'hand', 'good', 'like', 'sir', 'said', 'dick', 'ye']


The top 20 words for topic # 2
['way', 'answered', 'nt', 'took', 'shall', 'long', 'saw', 'princess', 'away', 'great', 'like', 'day', 'old', 'time', 'prince', 'came', 'went', 'little', 'king', 'said']


The top 20 words for topic # 3
['pilrig', 'corstorphine', 'unheard', 'rithmetic', 'jubal', 'locate', 'coverings', 'retorted', 'growing', 'summer', 'horns', 'cow', 'advocate', 'bossy', 'knobs', 'hanging', 'new', 'ones', 'rags', 'antlers']


The top 20 words for topic # 4
['way', 'children', 'mother', 'boys', 'michael', 'cried', 'kilmeny', 'know', 'darling', 'john', 'time', 'eric', 'nt', 'knew', 'hoo

In [26]:
# LDA / tf-idf
LDA = LatentDirichletAllocation(n_components = 10, n_jobs = -2, random_state = 42)

# fit the transfomed content with LDA
LDA.fit(words_tfidf)

# display the result
for index,topic in enumerate(LDA.components_):
    print(f"The top 20 words for topic # {index}")
    print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print("\n")

The top 20 words for topic # 0
['rosetta', 'advocate', 'eunice', 'gazelle', 'thyra', 'trevlyn', 'hoppyhippyhippyhopo', 'petru', 'really', 'coils', 'gluckstein', 'gibbet', 'executioner', 'thomson', 'lillian', 'waking', 'kitten', 'molinda', 'firedrake', 'remora']


The top 20 words for topic # 1
['theseus', 'mcdougal', 'falkenstein', 'swordbelt', 'henderland', 'cheque', 'jason', 'turretstair', 'fiske', 'ahmed', 'gluckstein', 'butler', 'tackleton', 'molinda', 'alphonso', 'enrico', 'maimie', 'benson', 'firedrake', 'daintyfoot']


The top 20 words for topic # 2
['tunstall', 'risingham', 'gloucester', 'alphonso', 'rankeillor', 'bennet', 'falkenstein', 'holywood', 'joanna', 'remora', 'hoseason', 'riach', 'shoreby', 'foxham', 'shelton', 'lawless', 'matcham', 'timmy', 'firedrake', 'daniel']


The top 20 words for topic # 3
['ferries', 'silverlaced', 'speers', 'lettervore', 'tenantry', 'herhe', 'henderland', 'envies', 'lettermore', 'lilly', 'enrico', 'waterpath', 'salome', 'gluckstein', 'spoonef